# Pb scavenging forcing fields

Pb is scavenged onto monthly lithogenic and biogenic particle fields 

$$ \frac{\partial [dPb]}{\partial t} = - (\beta \cdot P_{litho,authigenic} + (1-\beta) \cdot P_{biogenic} ) \cdot k_{p} \cdot [dPb] $$

In [1]:
import numpy as np
import xarray as xr
from scipy.spatial import Delaunay
from scipy import ndimage as nd
from scipy.interpolate import LinearNDInterpolator
import sys
sys.path.append('../paper-materials/')
from constants import imin, imax, jmin, jmax, isize, jsize

%matplotlib inline

#### Load files

In [2]:
# The Pb model configuration ANHA12 mask:
mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon     = mesh['nav_lon'].values
mesh_lat     = mesh['nav_lat'].values
mesh_bathy   = mesh['tmask'][0,:,:,:].values
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = mesh['nav_lev'].values

In [3]:
# ANHA4 BLING output 
# from https://canadian-nemo-ocean-modelling-forum-commuity-of-practice.readthedocs.io/en/latest/Institutions/UofA/Configurations/ANHA4/index.html
dset          = xr.open_dataset(f'/data/brogalla/ANHA4/BLING-EPM151/ANHA4-EPM151_y2002m08d13_gridB.nc')
ANHA4_lons    = dset['nav_lon'].values
ANHA4_lats    = dset['nav_lat'].values
ANHA4_depths  = dset['deptht'].values

In [4]:
# Mn for surface particle field:
dset_ANHA12 = xr.open_dataset('/data/brogalla/run_storage/Mn-extended-domain-202210/oMn_y2002m01.nc')
ANHA12_lons = mesh_lon[imin:imax,jmin:jmax]
ANHA12_lats = mesh_lat[imin:imax,jmin:jmax]

In [5]:
# interpolate from ANHA4 to ANHA12 grid:
tri = Delaunay(np.array([ANHA4_lons.flatten(), ANHA4_lats.flatten()]).transpose())  # Compute the triangulation

#### Functions:

In [6]:
# Replace the value of invalid data cells with the nearest valid cell (from a stackoverflow example)
def fill(data, invalid=None):

    if invalid is None: invalid = np.isnan(data)

    ind = nd.distance_transform_edt(invalid, return_distances=False, return_indices=True)
    return data[tuple(ind)]

In [7]:
# Save particle monthly particle fields to a netCDF file
def save_file_particle_fields(folder, oMn, fpop, fpops, biom, year, month):   
    
    file_write = xr.Dataset(
        {'oxidisMn' : (("deptht","y","x"), oMn),
         'fpop'     : (("deptht","y","x"), fpop),
         'fpop-sink': (("deptht","y","x"), fpops),
         'biomass'  : (("deptht","y","x"), biom)},
        coords = {
            "time_counter": np.zeros(1),
            "deptht": depths,
            "y": np.zeros(mesh_lat[imin:imax,jmin:jmax].shape[0]),
            "x": np.zeros(mesh_lon[imin:imax,jmin:jmax].shape[1])},
    )
    file_write.to_netcdf(f'{folder}Pb_scavenging_y{year}m{month:02}.nc', \
                         unlimited_dims='time_counter')

    return

In [8]:
# load monthly BLING files and calculate a biogenic particle field from it
def load_bio(year, month):
    file_EPM101  = f'/data/brogalla/ANHA4/BLING-EPM151/ANHA4-EPM151_y{year}m{month:02}.nc'
    
    # with the BLING file, calculate the fields:
    with xr.open_dataset(f'{file_EPM101}') as dset_bio:
        fpop_ANHA4   = dset_bio['fpop'][:,:,:].values      # sinking particulate organic matter in phosphate units (mol P/m3)
        fpop_ANHA4[fpop_ANHA4 < 0]       = 0
        fpop_ANHA4[np.isnan(fpop_ANHA4)] = np.nanmean(fpop_ANHA4)

        # Convert sinking particulate organic matter in phosphate units by dividing by the sinking rate:
        wsink              = np.ones(ANHA4_depths.shape)*16/(3600*24) # convert from m/day to m/s
        wsink[depths > 80] = (0.05*(ANHA4_depths[ANHA4_depths > 80]-80)+16)/(3600*24) 
        fpops_ANHA4        = np.array([fpop_ANHA4[d,:,:] / wsink[d] for d in range(0,len(ANHA4_depths))])
        fpops_ANHA4[fpop_ANHA4 < 0]       = 0
        fpops_ANHA4[np.isnan(fpop_ANHA4)] = np.nanmean(fpops_ANHA4)

        biop_ANHA4   = dset_bio['biomass_p'][:,:,:].values # biomass concentration in phosphate units (mol P/m3)
        biop_ANHA4[biop_ANHA4 < 0]       = 0
        biop_ANHA4[np.isnan(biop_ANHA4)] = np.nanmean(biop_ANHA4)
        
        return fpop_ANHA4, fpops_ANHA4, biop_ANHA4

# interpolate the particle field to the ANHA12 grid
def interpolate_bio(bio_particles):
    
    # interpolate from ANHA4 to ANHA12 grid:
    ANHA12_bio = np.empty((50,isize,jsize))
    for depth in range(0,50):
        interpolator   = LinearNDInterpolator(tri, bio_particles[depth,:,:].flatten())    
    
        ANHA12_bio[depth,:,:] = interpolator(np.array([ANHA12_lons.flatten(), ANHA12_lats.flatten()]).transpose()).reshape(ANHA12_lons.shape)

    return ANHA12_bio

# load the oxidised Mn fields from experiments with Rogalla et al. (2022) to use as a proxy for lithogenic particle fields
def load_litho(year, month):
    # Mn for surface particle field:
    folder_Mn   = f'/data/brogalla/run_storage/Mn-extended-domain-202210/'
    
    with xr.open_dataset(f'{folder_Mn}oMn_y{year}m{month:02}.nc') as dset_ANHA12:
        ANHA12_Mn   = dset_ANHA12['oxidismn'][:,:,:].values 

    return ANHA12_Mn

In [9]:
# main function that loads and interpolates the individual fields and creates particle fields to save to file
def create_particle_fields(year, month, save=False, loc=''):
    
    # load forcing fields and interpolate to ANHA12 grid
    ANHA4_fpop, ANHA4_fpops, ANHA4_biom = load_bio(year, month)
    ANHA12_fpop  = interpolate_bio(ANHA4_fpop)
    ANHA12_fpops = interpolate_bio(ANHA4_fpops)
    ANHA12_biom  = interpolate_bio(ANHA4_biom)
    ANHA12_litho = load_litho(year, month)

    # Fill any weird values 
    # https://stackoverflow.com/questions/3662361/fill-in-missing-values-with-nearest-neighbour-in-python-numpy-masked-arrays
    ANHA12_biom  = fill(ANHA12_biom,  invalid=(ANHA12_biom  < 0))
    ANHA12_fpop  = fill(ANHA12_fpop,  invalid=(ANHA12_fpop  < 0))
    ANHA12_fpops = fill(ANHA12_fpops, invalid=(ANHA12_fpops < 0))
    ANHA12_litho = fill(ANHA12_litho, invalid=(ANHA12_litho < 0))
    ANHA12_biom[np.isnan(ANHA12_biom)]   = 0.0; ANHA12_fpop[np.isnan(ANHA12_fpop)]   = 0.0;
    ANHA12_fpops[np.isnan(ANHA12_fpops)] = 0.0; ANHA12_litho[np.isnan(ANHA12_litho)] = 0.0;
    ANHA12_biom[(mesh_bathy[:,imin:imax,jmin:jmax] < 0.1)]  = 0.0; ANHA12_fpop[(mesh_bathy[:,imin:imax,jmin:jmax] < 0.1)]  = 0.0;
    ANHA12_litho[(mesh_bathy[:,imin:imax,jmin:jmax] < 0.1)] = 0.0; ANHA12_fpops[(mesh_bathy[:,imin:imax,jmin:jmax] < 0.1)] = 0.0; 
    
    if save:
        folder =f'/ocean/brogalla/GEOTRACES/data/Pb-forcing-202311/{loc}'
        save_file_particle_fields(f'{folder}', ANHA12_litho, ANHA12_fpop, ANHA12_fpops, ANHA12_biom, year, month)
    
    return ANHA12_litho, ANHA12_biom

Create particle fields and save:

In [ ]:
for year in range(2002, 2022):
    for month in range(1,13):
        print(year, month)
        litho, bio = create_particle_fields(year, month, save=True, loc = 'particle-fields/')

## Now combine these particle fields to create the scavenging forcing files:

In [10]:
# calculate the total scavenging particle field by combining all the individual fields
def calc_kscav(oMn, fpop, fpops, biom, oMn_max=np.nan, bio_max=np.nan,
               ln_fpop=False, ln_fpop_sink=True, ln_biomass=True, beta=0.005, power=1):
    # beta --- fraction lithogenic
    
    litho_particles_norm = oMn[:,:,:,:]/oMn_max
    bio_particles        = ln_fpop*fpop[:,:,:,:] + ln_fpop_sink*fpops[:,:,:,:] + ln_biomass*biom[:,:,:,:]
    
    bio_particles_norm   = bio_particles/(bio_max**(1-power))
    bio_particles_norm[bio_particles_norm < 0] = 0
    
    particles            = litho_particles_norm*beta + (1-beta)*(bio_particles_norm)
    part_scav            = particles
    return particles

In [18]:
# main scavenging calculation combines particle fields to a generalized particle field using the tuned parameter beta
def main_calc(year, bio_max=np.nan, oMn_max=np.nan, ln_fpop=False, ln_fpop_sink=True, ln_biomass=True):
    
    oMn   = np.zeros((12,len(depths),isize,jsize)); fpop  = np.zeros((12,len(depths),isize,jsize));
    fpops = np.zeros((12,len(depths),isize,jsize)); biom  = np.zeros((12,len(depths),isize,jsize));
    for month in range(1,13):
        df = xr.open_dataset(f'/ocean/brogalla/GEOTRACES/data/Pb-forcing-202311/particle-fields/Pb_scavenging_y{year}m{month:02}.nc')
        oMn[month-1,:,:,:]   = df['oxidisMn'].values[:,:,:]
        fpop[month-1,:,:,:]  = df['fpop'].values[:,:,:]
        fpops[month-1,:,:,:] = df['fpop-sink'].values[:,:,:]
        biom[month-1,:,:,:]  = df['biomass'].values[:,:,:]
        
    # calculate the particle field
    particles = calc_kscav(oMn, fpop, fpops, biom, bio_max=bio_max, oMn_max=oMn_max, beta=0.10, power=0.15)
    
    return particles

In [19]:
# calculate the maximum of the biogenic and lithogenic particle fields over time to use for normalization later
def calc_max(year, ln_fpop=False, ln_fpop_sink=True, ln_biomass=True):
    
    oMn   = np.zeros((12,len(depths),isize,jsize)); fpop  = np.zeros((12,len(depths),isize,jsize));
    fpops = np.zeros((12,len(depths),isize,jsize)); biom  = np.zeros((12,len(depths),isize,jsize));
    for month in range(1,13):
        # load the particle fields created in the upper section
        df = xr.open_dataset(f'/ocean/brogalla/GEOTRACES/data/Pb-forcing-202311/particle-fields/Pb_scavenging_y{year}m{month:02}.nc')
        oMn[month-1,:,:,:]   = df['oxidisMn'].values[:,:,:]
        fpop[month-1,:,:,:]  = df['fpop'].values[:,:,:]
        fpops[month-1,:,:,:] = df['fpop-sink'].values[:,:,:]
        biom[month-1,:,:,:]  = df['biomass'].values[:,:,:]

    litho_max = np.nanmax(oMn[:,:,:,:])    
    bio_max   = np.nanmax(ln_fpop*fpop[:,:,:,:] + ln_fpop_sink*fpops[:,:,:,:] + ln_biomass*biom[:,:,:,:])
    
    return litho_max, bio_max

In [16]:
# save the overall scavenging field to a netCDF file
def save_file_scavenging(folder, particle_array, year, month):   
    
    file_write = xr.Dataset(
        {'particles' : (("deptht","y","x"), particle_array)},
        coords = {
            "time_counter": np.zeros(1),
            "deptht": depths,
            "y": np.zeros(mesh_lat.shape[0]),
            "x": np.zeros(mesh_lon.shape[1])},
    )
    file_write.to_netcdf(f'{folder}Pb_scavenging_y{year}m{month:02}.nc', \
                         unlimited_dims='time_counter')

    return

In [ ]:
# find maximum values for biogenic and lithogenic particle fields:
litho_max = np.zeros(len(range(2002,2022)))
bio_max   = np.zeros(len(range(2002,2022)))
for ind, year in enumerate(range(2002,2022)):
    litho_max[ind], bio_max[ind] = calc_max(year)

In [ ]:
folder = '/ocean/brogalla/GEOTRACES/data/Pb-forcing-202311/scavenging/'
for year in range(2002,2022):
    particles                          = np.zeros((12,) + mesh_bathy.shape)
    particles[:,:,imin:imax,jmin:jmax] = main_calc(year, bio_max=7.47144e-4, oMn_max=1.83314e-07) # values from output from the previous cell
    for month in range(1,13): 
        save_file_scavenging(f'{folder}', particles[month-1,:,:,:], year, month)